In [1]:
import numpy as np
import pandas as pd

### Prediction

In [2]:
flow_pred = np.load('./results/20230713_033341_long_term_forecast_flow_dm256_df256_el2_topk5_nk6_fq_h_Exp_3/pred.npy')
speed_pred = np.load('./results/20230713_033319_long_term_forecast_speed_dm256_df256_el2_topk5_nk6_fq_h_Exp_3/pred.npy')

In [3]:
D, L, K = flow_pred.shape
flow_pred = flow_pred.reshape(D * L, K) # (D*L,K)
speed_pred = speed_pred.reshape(D * L, K) # (D*L,K)

In [4]:
# flow_pred_df = pd.DataFrame(data=flow_pred, index=datetime_5min_test_ids_full)
# speed_pred_df = pd.DataFrame(data=speed_pred, index=datetime_5min_test_ids_full)

# pred_time_range
pred_ranges_5min = [('08:00:00', '08:55:00'), ('12:30:00', '13:25:00'), ('17:00:00', '17:55:00')]

# Define the start and end dates for the DatetimeIndex
start_date = '2023-07-01'
end_date = '2023-07-07'

# Create an empty list to store the DatetimeIndex values
index_values = []

# Loop through each date
for date in pd.date_range(start=start_date, end=end_date, freq='D'):
    # Loop through each range and generate the DatetimeIndex values within that range for the current date
    for start_time, end_time in pred_ranges_5min:
        index_values += pd.date_range(start=f'{date.date()} {start_time}', end=f'{date.date()} {end_time}', freq='5min', inclusive='both').tolist()

datetime_5min_test_ids = pd.DatetimeIndex(index_values)

flow_pred_df = pd.DataFrame(data=flow_pred, index=datetime_5min_test_ids)
speed_pred_df = pd.DataFrame(data=speed_pred, index=datetime_5min_test_ids)

In [5]:
result_flow_pred = np.sum(flow_pred_df.values.reshape((252,10,4)), axis=2)
result_speed_pred = np.round(np.sum((flow_pred_df.values * speed_pred_df.values).reshape((252,10,4)), axis=2) / result_flow_pred, decimals=2)


template_df = pd.read_csv('./submit_template.csv',index_col=['Time', 'Week'])

num_columns = result_flow_pred.shape[1]


result_df = pd.DataFrame(columns=range(num_columns * 2), index=template_df.index)


for i in range(num_columns):
    result_df.iloc[:, 2 * i] = result_flow_pred[:, i]
    result_df.iloc[:, 2 * i + 1] = result_speed_pred[:, i]

result_df.columns = template_df.columns

num_columns = result_df.shape[1]

for i in range(num_columns):
    if i % 2 == 0:
        result_df.iloc[:, i] = result_df.iloc[:, i].round(0).astype(int)

C:\Users\great\AppData\Local\Temp\ipykernel_26092\580094491.py:14: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  result_df.iloc[:, 2 * i] = result_flow_pred[:, i]
C:\Users\great\AppData\Local\Temp\ipykernel_26092\580094491.py:15: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  result_df.iloc[:, 2 * i + 1] = result_speed_pred[:, i]
C:\Users\great\AppData\Local\Temp\ipykernel_26092\580094491.py:23: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To 

In [6]:
result_df.head(10)

Loop1 Flow (Veh/h)  Loop1 Speed (km/h)  Loop2 Flow (Veh/h)  \
Time Week                                                               
8:00 6                   2574          111.750000                2960   
8:05 6                   2742          109.449997                2645   
8:10 6                   2815          111.080002                2938   
8:15 6                   2866          109.080002                3214   
8:20 6                   2816          110.889999                3325   
8:25 6                   3097          108.190002                3280   
8:30 6                   2831          110.730003                3341   
8:35 6                   3044          110.449997                3493   
8:40 6                   3299          110.599998                3676   
8:45 6                   3614          109.510002                3822   

           Loop2 Speed (km/h)  Loop3 Flow (Veh/h)  Loop3 Speed (km/h)  \
Time Week                                                               
8:00 6             111.120003                2755          110.690002   
8:05 6             108.930000                2567          108.019997   
8:10 6             110.320000                2732          109.449997   
8:15 6             108.889999                3112          108.309998   
8:20 6             111.320000                2890          111.519997   
8:25 6             110.129997                3023          109.220001   
8:30 6             110.930000                3107          107.949997   
8:35 6             112.440002                3687          109.209999   
8:40 6             110.300003                3428          106.400002   
8:45 6             110.320000                3495          107.660004   

           Loop4 Flow (Veh/h)  Loop4 Speed (km/h)  Loop5 Flow (Veh/h)  \
Time Week                                                               
8:00 6                   3877          107.720001                2989   
8:05 6                   3892          109.589996                2716   
8:10 6                   3950          111.779999                2781   
8:15 6                   4674          105.220001                3413   
8:20 6                   4388          105.510002                3324   
8:25 6                   4047          110.489998                3064   
8:30 6                   4610          110.669998                2954   
8:35 6                   4777          104.629997                3251   
8:40 6                   4919          111.790001                3593   
8:45 6                   4925          108.290001                3493   

           Loop5 Speed (km/h)  Loop6 Flow (Veh/h)  Loop6 Speed (km/h)  \
Time Week                                                               
8:00 6             110.809998                3028          109.080002   
8:05 6             107.699997                2580          108.669998   
8:10 6             107.760002                2783          109.699997   
8:15 6             109.720001                3266          112.279999   
8:20 6             109.449997                3285          107.540001   
8:25 6             109.080002                3023          108.730003   
8:30 6             108.449997                3046          107.599998   
8:35 6             106.320000                3064          108.559998   
8:40 6             106.160004                3749          106.830002   
8:45 6             107.260002                3480          106.519997   

           Loop7 Flow (Veh/h)  Loop7 Speed (km/h)  Loop8 Flow (Veh/h)  \
Time Week                                                               
8:00 6                   3504          113.080002                3223   
8:05 6                   3183          110.730003                2672   
8:10 6                   3290          109.360001                2952   
8:15 6                   3680          109.709999                3123   
8:20 6                   3783          108.690002                

In [7]:
result_df.to_csv('./results/submit_3.csv', index=True)

### Imputation

In [2]:
flow_pred = np.load('./results/20230724_182914_TimesNet_imputation_flow_dm512_df512_el2_topk5_nk6_fq_t_Exp_2/pred.npy')
speed_pred = np.load('./results/20230724_202954_TimesNet_imputation_speed_dm512_df512_el2_topk5_nk6_fq_t_Exp_1/pred.npy')

In [4]:
D, L, K = flow_pred.shape
flow_pred = flow_pred.reshape(D * L, K) # (D*L,K)
speed_pred = speed_pred.reshape(D * L, K) # (D*L,K)

In [5]:
# Define the start and end times of each range
ranges_5min = [('05:00:00', '17:55:00')]

# Define the start and end dates for the DatetimeIndex
start_date = '2023-07-01'
end_date = '2023-07-07'

# Create an empty list to store the DatetimeIndex values
index_values = []

# Loop through each date
for date in pd.date_range(start=start_date, end=end_date, freq='D'):
    # Loop through each range and generate the DatetimeIndex values within that range for the current date
    for start_time, end_time in ranges_5min:
        index_values += pd.date_range(start=f'{date.date()} {start_time}', end=f'{date.date()} {end_time}', freq='5min', inclusive='both').tolist()

# Create the DatetimeIndex from the list of values
datetime_5min_test_ids_full = pd.DatetimeIndex(index_values)

flow_pred_df = pd.DataFrame(data=flow_pred, index=datetime_5min_test_ids_full)
speed_pred_df = pd.DataFrame(data=speed_pred, index=datetime_5min_test_ids_full)

In [7]:
# pred_time_range
pred_ranges_5min = [('08:00:00', '08:55:00'), ('12:30:00', '13:25:00'), ('17:00:00', '17:55:00')]

# Define the start and end dates for the DatetimeIndex
start_date = '2023-07-01'
end_date = '2023-07-07'

# Create an empty list to store the DatetimeIndex values
index_values = []

# Loop through each date
for date in pd.date_range(start=start_date, end=end_date, freq='D'):
    # Loop through each range and generate the DatetimeIndex values within that range for the current date
    for start_time, end_time in pred_ranges_5min:
        index_values += pd.date_range(start=f'{date.date()} {start_time}', end=f'{date.date()} {end_time}', freq='5min', inclusive='both').tolist()

datetime_5min_test_ids = pd.DatetimeIndex(index_values)

flow_pred_df = flow_pred_df.loc[datetime_5min_test_ids]
speed_pred_df = speed_pred_df.loc[datetime_5min_test_ids]

In [11]:
result_flow_pred = np.sum(flow_pred_df.values.reshape((252,10,4)), axis=2)
result_speed_pred = np.round(np.sum((flow_pred_df.values * speed_pred_df.values).reshape((252,10,4)), axis=2) / result_flow_pred, decimals=2)


template_df = pd.read_csv('./submit_template.csv',index_col=['Time', 'Week'])

num_columns = result_flow_pred.shape[1]


result_df = pd.DataFrame(columns=range(num_columns * 2), index=template_df.index)


for i in range(num_columns):
    result_df.iloc[:, 2 * i] = result_flow_pred[:, i]
    result_df.iloc[:, 2 * i + 1] = result_speed_pred[:, i]

result_df.columns = template_df.columns

num_columns = result_df.shape[1]

for i in range(num_columns):
    if i % 2 == 0:
        result_df.iloc[:, i] = result_df.iloc[:, i].round(0).astype(int)

C:\Users\great\AppData\Local\Temp\ipykernel_6072\580094491.py:14: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  result_df.iloc[:, 2 * i] = result_flow_pred[:, i]
C:\Users\great\AppData\Local\Temp\ipykernel_6072\580094491.py:15: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  result_df.iloc[:, 2 * i + 1] = result_speed_pred[:, i]
C:\Users\great\AppData\Local\Temp\ipykernel_6072\580094491.py:23: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To ret

In [12]:
result_df.head(10)

Loop1 Flow (Veh/h)  Loop1 Speed (km/h)  Loop2 Flow (Veh/h)  \
Time Week                                                               
8:00 6                   2502          112.410004                2565   
8:05 6                   2609          113.360001                2426   
8:10 6                   2692          115.480003                2629   
8:15 6                   3025          110.230003                3094   
8:20 6                   2876          107.580002                2953   
8:25 6                   2880          110.389999                2873   
8:30 6                   2902          110.849998                3241   
8:35 6                   3190          110.809998                3032   
8:40 6                   2856          110.370003                3606   
8:45 6                   3452          110.970001                4026   

           Loop2 Speed (km/h)  Loop3 Flow (Veh/h)  Loop3 Speed (km/h)  \
Time Week                                                               
8:00 6             112.690002                2353          112.660004   
8:05 6             112.529999                2210          115.309998   
8:10 6             110.010002                2673          110.330002   
8:15 6             113.680000                2719          109.449997   
8:20 6             111.209999                2649          108.419998   
8:25 6             112.500000                2932          110.080002   
8:30 6             113.540001                3101          110.599998   
8:35 6             110.750000                3189          107.730003   
8:40 6             115.639999                3023          108.739998   
8:45 6             110.389999                3688          109.110001   

           Loop4 Flow (Veh/h)  Loop4 Speed (km/h)  Loop5 Flow (Veh/h)  \
Time Week                                                               
8:00 6                   3446          108.519997                2372   
8:05 6                   3143          110.610001                2182   
8:10 6                   3303          109.879997                2418   
8:15 6                   3761          108.750000                2708   
8:20 6                   3605          106.040001                2530   
8:25 6                   3816          105.230003                2814   
8:30 6                   4212          107.150002                3242   
8:35 6                   4192          104.769997                2774   
8:40 6                   4266          107.040001                3011   
8:45 6                   4877          100.129997                3846   

           Loop5 Speed (km/h)  Loop6 Flow (Veh/h)  Loop6 Speed (km/h)  \
Time Week                                                               
8:00 6             112.160004                2668          113.160004   
8:05 6             120.889999                2460          123.309998   
8:10 6             110.080002                2689          114.449997   
8:15 6             108.260002                2825          103.250000   
8:20 6             105.139999                2883          103.410004   
8:25 6             101.360001                2989          104.599998   
8:30 6             104.809998                3141          105.500000   
8:35 6             103.699997                3219          104.430000   
8:40 6             110.709999                3206          115.779999   
8:45 6             104.739998                3980          111.320000   

           Loop7 Flow (Veh/h)  Loop7 Speed (km/h)  Loop8 Flow (Veh/h)  \
Time Week                                                               
8:00 6                   3067          110.070000                2579   
8:05 6                   2839          109.430000                2537   
8:10 6                   3102          109.040001                2691   
8:15 6                   3179          107.099998                2865   
8:20 6                   3143          107.879997                

In [13]:
result_df.to_csv('./results/submit_2.csv', index=True)